In [5]:
import cv2
import easyocr

def preprocess_image(image_path):
    """
    Simple image preprocessing for OCR
    
    Args:
        image_path (str): Path to input image
    
    Returns:
        numpy.ndarray: Preprocessed image
    """
    # Read the image
    img = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive thresholding
    binary = cv2.adaptiveThreshold(
        gray, 
        255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        11, 
        2
    )
    
    return binary

def recognize_text(image_path, languages=['hu', 'en']):
    """
    Recognize text with robust error handling
    
    Args:
        image_path (str): Path to input image
        languages (list): Languages to use for OCR
    
    Returns:
        list: Detected text lines
    """
    # Initialize reader
    reader = easyocr.Reader(
        languages, 
        gpu=False  # Explicit CPU mode
    )
    
    # Preprocessing strategies
    preprocessing_methods = [
        lambda img: img,  # Original image
        preprocess_image,  # Custom preprocessing
        lambda img: cv2.equalizeHist(img)  # Histogram equalization
    ]
    
    # Try different preprocessing methods
    for preprocess_func in preprocessing_methods:
        try:
            # Preprocess the image
            processed_img = preprocess_func(image_path)
            
            # Try different OCR configurations
            ocr_configs = [
                {'detail': 0},  # Simple text extraction
                {'detail': 1}   # Detailed extraction
            ]
            
            for config in ocr_configs:
                try:
                    # Attempt text recognition
                    results = reader.readtext(
                        processed_img, 
                        detail=config['detail']
                    )
                    
                    # Normalize results
                    if results:
                        # Extract text, handling different result formats
                        extracted_texts = []
                        for result in results:
                            # Handle different possible result structures
                            if isinstance(result, str):
                                extracted_texts.append(result)
                            elif isinstance(result, (list, tuple)):
                                # Extract text from different index positions
                                text_index = -1 if config['detail'] == 1 else 0
                                text = result[text_index]
                                extracted_texts.append(str(text))
                        
                        # Return non-empty results
                        if extracted_texts:
                            return extracted_texts
                
                except Exception as config_error:
                    print(f"OCR Configuration Error: {config_error}")
        
        except Exception as preprocess_error:
            print(f"Preprocessing Error: {preprocess_error}")
    
    # If all methods fail
    return []

def main():
    # Path to your Hungarian text image
    image_path = 'cropped_rectangle.jpg'
    
    try:
        # Recognize text
        results = recognize_text(image_path)
        
        # Print results
        print("Detected Text:")
        for text in results:
            print(text)
    
    except Exception as e:
        print(f"OCR Processing Error: {e}")

if __name__ == "__main__":
    # Prerequisites:
    # pip install easyocr opencv-python
    main()

Using CPU. Note: This module is much faster with a GPU.


Detected Text:
Bukaza flams ~i Reoxaa Jcén
